#### import the required libs (numpy and pandas)

In [1]:
import pandas as pd
import numpy as np
pd.set_option('expand_frame_repr', False)

#### load csvs to pandas dataframes, we will use online data for the best accuracy

In [2]:
search_trend_url ="https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/search_trends_symptoms_dataset/United%20States%20of%20America/2020_US_weekly_symptoms_dataset.csv"
hospitalization_url ="https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/cc_by/aggregated_cc_by.csv"
search_trend_df = pd.read_csv(search_trend_url)
hospitalization_df = pd.read_csv(hospitalization_url)


### We will try to clean the search trend dataset first

after loading the dataset to pandas dataframe, we will check how many parameters does the dataframe contains

In [3]:
len(search_trend_df.columns.values)

430

#### Since we have 430 parameters, we now examine the data, and try to determine if we can drop some of the columns to avoid over-fitting

In [4]:
search_trend_df.iloc[np.r_[0:10,-10:0]]

,open_covid_region_code,country_region_code,country_region,sub_region_1,sub_region_1_code,sub_region_2,sub_region_2_code,date,symptom:Abdominal obesity,symptom:Abdominal pain,...,symptom:Wart,symptom:Water retention,symptom:Weakness,symptom:Weight gain,symptom:Wheeze,symptom:Xeroderma,symptom:Xerostomia,symptom:Yawn,symptom:hyperhidrosis,symptom:pancreatitis
0,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-01-06,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.28,NaN,NaN
1,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-01-13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.26,NaN,NaN
2,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-01-20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.48,NaN,NaN
3,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-01-27,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.93,NaN,NaN
4,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-02-03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.93,NaN,NaN
5,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-02-10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.05,NaN,NaN
6,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-02-17,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.62,NaN,NaN
7,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-02-24,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.42,NaN,NaN
8,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-03-02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.30,NaN,NaN
9,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-03-09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.77,NaN,NaN


Since we have several NaN columns, we can drop the columns where all data in the column are NaN

And also, by looking at the dataset, columns named as `country_region_code`, `country_region`, and `open_covid_region_code` can be dropped.

In [5]:
search_trend_df = search_trend_df.drop(search_trend_df.columns[[0,1,2,4,5,6]], axis=1)
search_trend_df = search_trend_df.dropna(axis=1, how='all')
search_trend_df = search_trend_df.rename(columns={'sub_region_1':'region_name'})
search_trend_df


,region_name,date,symptom:Adrenal crisis,symptom:Ageusia,symptom:Allergic conjunctivitis,symptom:Amblyopia,symptom:Amenorrhea,symptom:Angular cheilitis,symptom:Anosmia,symptom:Aphonia,...,symptom:Tenderness,symptom:Thyroid nodule,symptom:Trichoptilosis,symptom:Upper respiratory tract infection,symptom:Urethritis,symptom:Urinary urgency,symptom:Vasculitis,symptom:Ventricular fibrillation,symptom:Viral pneumonia,symptom:Yawn
0,Alaska,2020-01-06,12.69,NaN,NaN,NaN,12.29,NaN,NaN,NaN,...,14.28,15.86,11.90,18.84,13.88,NaN,14.08,9.91,NaN,14.28
1,Alaska,2020-01-13,9.56,NaN,NaN,NaN,12.62,NaN,NaN,NaN,...,13.39,12.24,7.84,16.83,13.01,7.65,15.30,8.41,NaN,16.26
2,Alaska,2020-01-20,NaN,NaN,NaN,NaN,14.17,NaN,7.36,NaN,...,12.88,17.66,15.27,18.40,12.14,7.54,10.30,16.19,NaN,17.48
3,Alaska,2020-01-27,15.31,7.47,NaN,NaN,15.67,7.47,NaN,7.65,...,13.30,12.76,9.84,20.41,17.49,NaN,11.30,NaN,NaN,10.93
4,Alaska,2020-02-03,8.81,NaN,NaN,NaN,13.87,8.06,NaN,12.56,...,11.81,16.68,10.68,20.62,16.87,NaN,14.24,10.68,10.5,18.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,Wyoming,2020-08-31,5.03,2.79,NaN,5.75,4.19,NaN,NaN,NaN,...,3.07,3.07,2.57,2.68,NaN,NaN,5.08,NaN,NaN,3.63
620,Wyoming,2020-09-07,3.36,3.25,2.2,3.63,NaN,2.25,2.70,NaN,...,4.29,2.81,NaN,3.19,3.19,NaN,4.51,NaN,NaN,4.02
621,Wyoming,2020-09-14,4.15,4.97,NaN,5.14,5.79,NaN,4.59,NaN,...,3.93,5.63,3.06,4.59,3.22,2.68,3.88,NaN,NaN,4.37
622,Wyoming,2020-09-21,5.64,3.67,NaN,5.25,3.77,NaN,5.09,NaN,...,3.28,3.72,3.17,3.23,2.90,NaN,5.14,3.23,NaN,4.76


now, let's check the dataset again

In [6]:
search_trend_df.iloc[np.r_[0:10,-10:0]]

,region_name,date,symptom:Adrenal crisis,symptom:Ageusia,symptom:Allergic conjunctivitis,symptom:Amblyopia,symptom:Amenorrhea,symptom:Angular cheilitis,symptom:Anosmia,symptom:Aphonia,...,symptom:Tenderness,symptom:Thyroid nodule,symptom:Trichoptilosis,symptom:Upper respiratory tract infection,symptom:Urethritis,symptom:Urinary urgency,symptom:Vasculitis,symptom:Ventricular fibrillation,symptom:Viral pneumonia,symptom:Yawn
0,Alaska,2020-01-06,12.69,NaN,NaN,NaN,12.29,NaN,NaN,NaN,...,14.28,15.86,11.90,18.84,13.88,NaN,14.08,9.91,NaN,14.28
1,Alaska,2020-01-13,9.56,NaN,NaN,NaN,12.62,NaN,NaN,NaN,...,13.39,12.24,7.84,16.83,13.01,7.65,15.30,8.41,NaN,16.26
2,Alaska,2020-01-20,NaN,NaN,NaN,NaN,14.17,NaN,7.36,NaN,...,12.88,17.66,15.27,18.40,12.14,7.54,10.30,16.19,NaN,17.48
3,Alaska,2020-01-27,15.31,7.47,NaN,NaN,15.67,7.47,NaN,7.65,...,13.30,12.76,9.84,20.41,17.49,NaN,11.30,NaN,NaN,10.93
4,Alaska,2020-02-03,8.81,NaN,NaN,NaN,13.87,8.06,NaN,12.56,...,11.81,16.68,10.68,20.62,16.87,NaN,14.24,10.68,10.50,18.93
5,Alaska,2020-02-10,11.97,NaN,NaN,NaN,11.97,NaN,NaN,10.88,...,9.97,15.05,12.51,18.68,14.69,7.44,9.07,NaN,7.80,17.05
6,Alaska,2020-02-17,9.16,NaN,NaN,NaN,9.33,NaN,NaN,9.16,...,22.56,14.42,10.18,16.96,14.93,NaN,13.57,8.65,8.48,16.62
7,Alaska,2020-02-24,10.44,6.63,NaN,NaN,11.44,10.61,9.78,7.29,...,11.77,17.57,15.58,18.56,10.44,7.13,12.26,8.62,11.10,14.42
8,Alaska,2020-03-02,14.62,NaN,NaN,NaN,9.97,NaN,NaN,NaN,...,10.80,19.43,12.96,21.76,11.29,NaN,13.29,10.46,14.12,9.30
9,Alaska,2020-03-09,10.60,NaN,11.61,NaN,12.79,NaN,7.07,7.91,...,12.96,11.11,13.97,46.95,10.27,NaN,9.26,NaN,18.85,10.77


Thus, we reduce the number of columns from 430 to 123

Yet, we still see a lot of NaNs, let's check the number of NaNs in each column

In [7]:
search_trend_df.isnull().sum()

region_name                           0
date                                  0
symptom:Adrenal crisis              355
symptom:Ageusia                     378
symptom:Allergic conjunctivitis     287
                                   ... 
symptom:Urinary urgency             224
symptom:Vasculitis                  549
symptom:Ventricular fibrillation    193
symptom:Viral pneumonia             224
symptom:Yawn                        547
Length: 123, dtype: int64

We will determine if we need to deal with these during training

### Now, it's time to clean the hospitalization dataset

We still need to have a look at the dataset first

In [8]:
hospitalization_df

,open_covid_region_code,region_name,date,cases_cumulative,cases_new,cases_cumulative_per_million,cases_new_per_million,deaths_cumulative,deaths_new,deaths_cumulative_per_million,...,stringency_index,stringency_index_for_display,stringency_legacy_index,stringency_legacy_index_for_display,government_response_index,government_response_index_for_display,containment_health_index,containment_health_index_for_display,economic_support_index,economic_support_index_for_display
0,AFG,Afghanistan,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
2,AFG,Afghanistan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
3,AFG,Afghanistan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
4,AFG,Afghanistan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98941,VUT,Vanuatu,2020-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.22,22.22,28.57,28.57,25.64,25.64,25.76,25.76,25.0,25.0
98942,VUT,Vanuatu,2020-10-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22.22,NaN,28.57,NaN,25.64,NaN,25.76,NaN,25.0
98943,VUT,Vanuatu,2020-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22.22,NaN,28.57,NaN,25.64,NaN,25.76,NaN,25.0
98944,VUT,Vanuatu,2020-10-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22.22,NaN,28.57,NaN,25.64,NaN,25.76,NaN,25.0


Since we only need `open_covid_region_code`, `region_name`,  `date`, and `hospitalized_new`, we will select only these columns

And in this mini-project only, we will use only data gathered from US


In [9]:
hospitalization_df = hospitalization_df[['open_covid_region_code','region_name','date','hospitalized_new']]
hospitalization_df = hospitalization_df[hospitalization_df['open_covid_region_code'].str.contains('^US-')]
hospitalization_df = hospitalization_df.drop(hospitalization_df.columns[[0]], axis=1)
regions = hospitalization_df.region_name.unique()

for region in regions:
    region_df = hospitalization_df[hospitalization_df['region_name'] == region]
    if (region_df['hospitalized_new'] == 0).all():
        hospitalization_df = hospitalization_df.drop(hospitalization_df[hospitalization_df.region_name == region].index, axis=0) 
hospitalization_df

,region_name,date,hospitalized_new
85278,Wyoming,2020-03-07,0.0
85279,Wyoming,2020-03-08,0.0
85280,Wyoming,2020-03-09,0.0
85281,Wyoming,2020-03-10,0.0
85282,Wyoming,2020-03-11,0.0
...,...,...,...
97526,Alaska,2020-10-04,0.0
97527,Alaska,2020-10-05,0.0
97528,Alaska,2020-10-06,0.0
97529,Alaska,2020-10-07,0.0


Since the data here are daily basis, and the data in search_trend_df are weekly basis, we need to convert these to weekly basis

In [10]:
hospitalization_df['date'] = pd.to_datetime(hospitalization_df['date'], format='%Y-%m-%d')
hospitalization_df = hospitalization_df.groupby(['region_name',]).resample('W', on='date',loffset='1d').sum()
hospitalization_df = hospitalization_df.reset_index()
hospitalization_df

,region_name,date,hospitalized_new
0,Alabama,2020-03-09,0.0
1,Alabama,2020-03-16,0.0
2,Alabama,2020-03-23,0.0
3,Alabama,2020-03-30,0.0
4,Alabama,2020-04-06,231.0
...,...,...,...
1354,Wyoming,2020-09-14,9.0
1355,Wyoming,2020-09-21,15.0
1356,Wyoming,2020-09-28,19.0
1357,Wyoming,2020-10-05,20.0


now the cleanning is done, we merge the datasets

We also drop column `open_covid_region_code` since we already have `sub_region_1` 

For future purpose, we rename `sub_region_1` to `region_name`

In [11]:
search_trend_df['date'] = pd.to_datetime(search_trend_df['date'], format='%Y-%m-%d')
result_df = search_trend_df.merge(hospitalization_df, on=['region_name', 'date'])
result_df

,region_name,date,symptom:Adrenal crisis,symptom:Ageusia,symptom:Allergic conjunctivitis,symptom:Amblyopia,symptom:Amenorrhea,symptom:Angular cheilitis,symptom:Anosmia,symptom:Aphonia,...,symptom:Thyroid nodule,symptom:Trichoptilosis,symptom:Upper respiratory tract infection,symptom:Urethritis,symptom:Urinary urgency,symptom:Vasculitis,symptom:Ventricular fibrillation,symptom:Viral pneumonia,symptom:Yawn,hospitalized_new
0,Alaska,2020-03-09,10.60,NaN,11.61,NaN,12.79,NaN,7.07,7.91,...,11.11,13.97,46.95,10.27,NaN,9.26,NaN,18.85,10.77,0.0
1,Alaska,2020-03-16,11.69,NaN,6.39,NaN,NaN,NaN,15.59,7.95,...,14.49,9.04,78.71,12.00,NaN,11.53,NaN,24.94,14.34,0.0
2,Alaska,2020-03-23,11.15,16.57,NaN,NaN,9.53,NaN,45.17,NaN,...,7.63,8.36,37.54,8.80,NaN,6.75,7.04,13.79,15.69,0.0
3,Alaska,2020-03-30,8.96,8.96,7.11,NaN,8.53,NaN,19.05,NaN,...,9.38,14.79,24.46,9.81,7.96,7.54,7.11,8.82,17.63,6.0
4,Alaska,2020-04-06,10.08,7.35,NaN,NaN,10.80,7.06,6.77,NaN,...,8.64,13.83,16.28,10.08,NaN,11.09,NaN,9.07,14.12,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,Wyoming,2020-08-31,5.03,2.79,NaN,5.75,4.19,NaN,NaN,NaN,...,3.07,2.57,2.68,NaN,NaN,5.08,NaN,NaN,3.63,8.0
417,Wyoming,2020-09-07,3.36,3.25,2.20,3.63,NaN,2.25,2.70,NaN,...,2.81,NaN,3.19,3.19,NaN,4.51,NaN,NaN,4.02,4.0
418,Wyoming,2020-09-14,4.15,4.97,NaN,5.14,5.79,NaN,4.59,NaN,...,5.63,3.06,4.59,3.22,2.68,3.88,NaN,NaN,4.37,9.0
419,Wyoming,2020-09-21,5.64,3.67,NaN,5.25,3.77,NaN,5.09,NaN,...,3.72,3.17,3.23,2.90,NaN,5.14,3.23,NaN,4.76,15.0


We may need to use the dataframe in another file, so we export it as a CSV file

And also, we do not want to have index in our generated CSV file, so we use `index=False`

In [12]:
result_df.to_csv('result.csv', index=False)

### The cleaning process is done